Here we will enhance our agent by adding tool-calling capabilities, allowing it to interact with external functions dynamically

Challenge:

Build an AI agent that:

    - Fecthes real-time stock prices
    - Performs some complex calculations
    - Queries a weeather API
    - Searches a Database

Instead of manually calling a function, our agent will automatically detect when a tool is needed and invoke it

import all the necessary libraries

In [2]:
import json
import datetime
import inspect
import requests
from typing import (TypedDict, List, Dict, Literal, Callable, Optional, Any, get_type_hints)
from openai import OpenAI
from openai.types.chat.chat_completion_message import ChatCompletionMessage
from openai.types.chat.chat_completion_message_tool_call import ChatCompletionMessageToolCall

Instantiate the OpenAI client with our environement variable openai API key

In [3]:
from dotenv import load_dotenv
load_dotenv()
client = OpenAI()

Recap our Memory & Function

Combine memory with custom python functions to create the full cycle of tool calling, which enabled an LLM to interact with the world

Class: a class is a blueprint for creating objects. An object is a self-contained unit of data and functionality. A class allows us to bundle related state and behavior under one name and then instantiate that blueprint to produce many independent objects with the same structure.

In [4]:
class Memory:
    def __init__(self):
        self._messages: List[Dict[str, str]] = []

    def add_message(self, 
        role: Literal['user', 'system', 'assistant', 'tool'], 
        content: str,
        tool_calls: dict=dict(),
        tool_call_id=None)->None:

        message = {
            "role": role,
            "content": content,
            "tool_calls": tool_calls,
        }

        if role == "tool":
            message = {
                "role": role,
                "content": content,
                "tool_call_id": tool_call_id,
            }

        self._messages.append(message)
    
    def get_messages(self) -> List[Dict[str, str]]:
        return self._messages
    
    def last_message(self) -> None:
        if self._messages:
            return self._messages[-1]

    def reset(self) -> None:
        self._messages = []

In [5]:
def chat_with_tools(user_question:str=None,
        memory:Memory=None,
        model:str="gpt-4o-mini",
        temperature=0.0,
        tools=None)-> str:
    messages = [{"role": "user", "content": user_question}]
    if memory:
        if user_question:
            memory.add_message(role="user", content=user_question)
        messages = memory.get_messages()

    response = client.chat.completions.create(
        model = model,
        temperature = temperature,
        messages = messages,
        tools = tools,
    )

    ai_message = str(response.choices[0].message.content)
    tool_calls = response.choices[0].message.tool_calls

    if memory:
        memory.add_message(role="assistant", content=ai_message, tool_calls=tool_calls)

    return ai_message

In [6]:
def power(base:float, exponent:float):
    """Exponentiation: base to the power of exponent"""
    return base ** exponent

In [7]:
tools = [{
    "type": "function",
    "function": {
        "name": "power",
        "description": "Exponentiation: base to the power of exponent",
        "parameters": {
            "type": "object",
            "properties": {
                "base": {"type": "number"},
                "exponent": {"type": "number"}
            },
            "required": ["base", "exponent"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

Next lets instantiate memory and run with the system prompt

In [8]:
memory = Memory()
memory.add_message(role="system", content="Youre a helpful assistant")

Call the LLM with a question that needs a tool

In [9]:
ai_message = chat_with_tools(
    "what is 2 to the power of -5?",
    model="gpt-4o-mini",
    memory=memory,
    tools=tools,
)

Get the arguments from the tool_calls object and call the actual defined function

In [10]:
args = json.loads(memory.last_message()['tool_calls'][0].function.arguments)
result = power(args["base"], args["exponent"])

Extract the tool_call_id and feed the LLM with the result from the function

In [11]:
tool_call_id = memory.last_message()['tool_calls'][0].id
memory.add_message(role="tool", content=str(result), tool_call_id=tool_call_id)
ai_message = chat_with_tools(
    model="gpt-4o-mini",
    tools=tools,
    memory=memory,
)

In [12]:
memory.get_messages()

[{'role': 'system', 'content': 'Youre a helpful assistant', 'tool_calls': {}},
 {'role': 'user',
  'content': 'what is 2 to the power of -5?',
  'tool_calls': {}},
 {'role': 'assistant',
  'content': 'None',
  'tool_calls': [ChatCompletionMessageToolCall(id='call_lGO2oNuWRzb5FXJaed7CTu26', function=Function(arguments='{"base":2,"exponent":-5}', name='power'), type='function')]},
 {'role': 'tool',
  'content': '0.03125',
  'tool_call_id': 'call_lGO2oNuWRzb5FXJaed7CTu26'},
 {'role': 'assistant',
  'content': '\\( 2 \\) to the power of \\(-5\\) is \\( 0.03125 \\).',
  'tool_calls': None}]

Now we will create our tool abstractions

Currently the way we've built our tools is prone to errors.

The task will be to create an abstraciton to make it easier to build a tool and call it

Look at the JSON schema we made for the power tool and think about the other following tools:

The tool class should at least have the following methods

    - __init__() : receiving the function and some logic to extract docs, arguments and their types
    - dict() : to return the JSON schema
    - __call__() : this enables the object that gets instantiated to be callable


More hints:
-by using get_type_hints(power)
    - if we run this we will get an output of the needed types of data to pass
-inspect.signature()
    -

In [26]:
class Tool:
    def __init__(self, func:Callable):
        self.func = func
        self.name = func.__name__
        self.description = func.__doc__
        self.arguments_types_map = get_type_hints(func)
        self.signature = inspect.signature(func)
        self.arguments = [
            {
                "name": key,
                "type":self._infer_json_schema_type(value),
                "required": param.default == inspect.Parameter.empty
            }
            for key, value in self.arguments_types_map.items()
            if (param := self.signature.parameters.get(key))
        ]
    def _infer_json_schema_type(self, arg_type: Any) -> str:
        if arg_type == bool:
            return "boolean"
        elif arg_type == int:
            return "integer"
        elif arg_type == float:
            return "number"
        elif arg_type == str:
            return "string"
        elif arg_type == list:
            return "array"
        elif arg_type == dict:
            return "object"
        elif arg_type == None:
            return "Null"
        elif arg_type == datetime.date or arg_type == datetime.datetime:
            return "string"
        else:
            return "string"

        
    def dict(self):
        return{
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parallel_tool_calls": False,
                "parameters": {
                    "type": "object",
                    "properties": {
                        argument["name"]: {
                            "type": argument["type"],
                        }
                        for argument in self.arguments
                    },
                    "required": [
                        argument["name"]
                        for argument in self.arguments
                        if argument["required"]
                    ],
                    "additionalProperties": False
                },
                "strict": True
            }
        }
    def __call__ (self, *args, **kwargs):
        return self.func(*args, **kwargs)


Lets recreate our power function to see if it works with our tool

In [27]:
def power(base:float, exponent:float):
    """Exponentiation: base to the power of exponent"""
    return base ** exponent

In [28]:
power_tool = Tool(power)

In [29]:
power_tool.dict()

{'type': 'function',
 'function': {'name': 'power',
  'description': 'Exponentiation: base to the power of exponent',
  'parallel_tool_calls': False,
  'parameters': {'type': 'object',
   'properties': {'base': {'type': 'number'}, 'exponent': {'type': 'number'}},
   'required': ['base', 'exponent'],
   'additionalProperties': False},
  'strict': True}}

In [30]:
power_tool(2,3)

8

In [32]:
class Agent:
    """ A Tool calling AI agent """

    def __init__ (
        self,
        name:str = "Agent",
        role:str = "Personal Assistant",
        instructions:str = "Help users with any questions",
        model:str = "gpt-4o-mini",
        temperature:float = 0.0,
        tools:List[Tool] = [],
    ):
        self.name = name
        self.role = role
        self.instructions = instructions
        self.model = model
        self.temperature = temperature
        self.memory = Memory()
        self.memory.add_message(
            role="system",
            content=f"You're an AI agent your role is {self.role}, "
                    f"and you need to {self.instructions}.",
        )

        self.client = OpenAI()


        self.tools = tools
        self.tool_map = {t.name:t for t in tools}
        self.openai_tools = [t.dict() for t in self.tools] if self.tools else None
    

    def invoke(self, user_message: str) -> str:
        self.memory.add_message(
            role="user",
            content=user_message
        )

        ai_message = self._get_completion(
            messages = self.memory.get_messages(),
        )

        tool_calls = ai_message.tool_calls
        self.memory.add_message(
            role= "assistant",
            content=ai_message.content,
            tool_calls = tool_calls,
        )

        if tool_calls:
            self._call_tools(tool_calls)

        return self.memory.last_message()


    def _call_tools(self, tool_calls:List[ChatCompletionMessageToolCall]):
        for t in tool_calls:
            tool_call_id = t.id
            function_name = t.function.name
            args = json.loads(t.function.arguments)
            callable_tool = self.tool_map[function_name]
            result = callable_tool(**args)
            self.memory.add_message(
                role="tool",
                content=str(result),
                tool_call_id=tool_call_id
            )
        
        ai_message = self._get_completion(
            messages = self.memory.get_messages(),
        )

        tool_calls = ai_message.tool_calls

        self.memory.add_message(
            role="assistant",
            content=ai_message.content,
            tool_calls = tool_calls,
        )

        if tool_calls:
            self.call_tools(tool_calls)


    def _get_completion(self, messages:List[Dict])-> ChatCompletionMessage:
        response = self.client.chat.completions.create(
            model = self.model,
            temperature = self.temperature,
            messages = messages,
            tools = self.openai_tools,
        )

        return response.choices[0].message

BUILD SOME AGENTS NOW   

In [35]:
agent = Agent(
    tools = [Tool(power)]
)

In [36]:
agent.invoke("what is nine to the power of two?")

{'role': 'assistant',
 'content': 'Nine to the power of two is 81.',
 'tool_calls': None}

In [37]:
agent.invoke("what is six + seven?")

{'role': 'assistant', 'content': 'Six plus seven is 13.', 'tool_calls': None}

In [39]:
agent.invoke("what is 2 to the power of -5?")

{'role': 'assistant',
 'content': 'Two to the power of negative five is 0.03125.',
 'tool_calls': None}

In [38]:
agent.invoke("what is 10 to the 3?")

{'role': 'assistant',
 'content': 'Ten to the power of three is 1000.',
 'tool_calls': None}

In [40]:
agent.memory.get_messages()

[{'role': 'system',
  'content': "You're an AI agent your role is Personal Assistant, and you need to Help users with any questions.",
  'tool_calls': {}},
 {'role': 'user',
  'content': 'what is nine to the power of two?',
  'tool_calls': {}},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [ChatCompletionMessageToolCall(id='call_9GxNNIvAyAeJ8HIdFQykw3cI', function=Function(arguments='{"base":9,"exponent":2}', name='power'), type='function')]},
 {'role': 'tool',
  'content': '81',
  'tool_call_id': 'call_9GxNNIvAyAeJ8HIdFQykw3cI'},
 {'role': 'assistant',
  'content': 'Nine to the power of two is 81.',
  'tool_calls': None},
 {'role': 'user', 'content': 'what is six + seven?', 'tool_calls': {}},
 {'role': 'assistant', 'content': 'Six plus seven is 13.', 'tool_calls': None},
 {'role': 'user', 'content': 'what is 10 to the 3?', 'tool_calls': {}},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [ChatCompletionMessageToolCall(id='call_IPMwcGTdxqJDbH3lzcbaZBWz', function

Explanation:



In [41]:
agent.memory.reset()

In [42]:
agent.memory.get_messages()

[]

In [43]:
agent.invoke("what is 10 to the 1?")

{'role': 'assistant',
 'content': '10 to the power of 1 is 10.',
 'tool_calls': None}

In [44]:
agent.memory.get_messages()

[{'role': 'user', 'content': 'what is 10 to the 1?', 'tool_calls': {}},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [ChatCompletionMessageToolCall(id='call_jleNNXtm4CixKJQMcQXdYTpD', function=Function(arguments='{"base":10,"exponent":1}', name='power'), type='function')]},
 {'role': 'tool',
  'content': '10',
  'tool_call_id': 'call_jleNNXtm4CixKJQMcQXdYTpD'},
 {'role': 'assistant',
  'content': '10 to the power of 1 is 10.',
  'tool_calls': None}]

In [45]:
agent.memory.reset()

In [46]:
agent.memory.get_messages()

[]

In [47]:
agent.invoke("what is 6.75 to the power of -5?")

{'role': 'assistant',
 'content': '\\( 6.75 \\) to the power of \\(-5\\) is approximately \\( 0.00007136432064128647 \\) or \\( 7.14 \\times 10^{-5} \\).',
 'tool_calls': None}

In [48]:
agent.memory.get_messages()

[{'role': 'user',
  'content': 'what is 6.75 to the power of -5?',
  'tool_calls': {}},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [ChatCompletionMessageToolCall(id='call_2IHUNKTg5s7hwWn3YSNNZgjt', function=Function(arguments='{"base":6.75,"exponent":-5}', name='power'), type='function')]},
 {'role': 'tool',
  'content': '7.136432064128647e-05',
  'tool_call_id': 'call_2IHUNKTg5s7hwWn3YSNNZgjt'},
 {'role': 'assistant',
  'content': '\\( 6.75 \\) to the power of \\(-5\\) is approximately \\( 0.00007136432064128647 \\) or \\( 7.14 \\times 10^{-5} \\).',
  'tool_calls': None}]